# nn.Module로 구현하는 선형 회귀

파이토치에서 이미 구현되어져 있는 함수들을 불러와서 선형 회귀 모델을 구현할 수 있다. 예를 들어 파이토치에서는 선형 회귀 모델이 nn.Linear()라는 함수로, 평균 제곱 오차가 nn.functional.mse_loss()라는 함수로 구현되어져 있다.

```python
import torch.nn as nn
model = nn.Linear(input_dim, output_dim)

import torch.nn.functional as F
cost = F.mse_loss(prediction, y_train)
```

## 단순 선형 회귀 구현

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
torch.manual_seed(1)

$ W = 2, b = 0 $ 임을 알고 데이터 선언

In [18]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

선형 회귀 모델 구현
- nn.Linear()는 입력의 차원과 출력의 차원을 인수로 받는다.

In [19]:
input_dim = 1
output_dim = 1
model = nn.Linear(input_dim, output_dim)

In [20]:
x_train.size(), y_train.size()

(torch.Size([3, 1]), torch.Size([3, 1]))

x_train, y_train 모두 1차원이므로 1을 각 인자로 설정한다. model에는 가중치 W와 편향 b가 저장되어 있고 이를 parameters() 함수를 사용해서 불러올 수 있다.

In [21]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.8742]], requires_grad=True), Parameter containing:
tensor([0.3112], requires_grad=True)]


첫번째 값이 가중치 W이고 두번째 값이 편향 b이다. 두 값 모두 랜덤한 값으로 초기화되어 있고 학습을 위해 requires_grad 인자가 True 값으로 설정되어있다.

In [22]:
print(list(nn.Linear(3, 2).parameters()))

[Parameter containing:
tensor([[-0.2150, -0.3487, -0.0968],
        [-0.2490, -0.1850,  0.0276]], requires_grad=True), Parameter containing:
tensor([0.3442, 0.3138], requires_grad=True)]


입출력 차원에 따라 가중치와 편향의 크기가 달라지는 것을 알 수 있다.

In [23]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

최적화 알고리즘(옵티마이저)를 정의한다. 앞에서는 직접 가중치와 편향을 리스트로 전달했지만 model의 parameters 함수로 전달할 수 있다. 사용하는 알고리즘은 SGD(경사하강법)이고 학습률은 0.01로 정의했다.

In [24]:
EPOCHS = 2000
for epoch in range(1, EPOCHS+1):
    pred = model(x_train)
    cost = nn.functional.mse_loss(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"epoch: {epoch:4d}/{EPOCHS} | cost: {cost:.8f}")

epoch:  100/2000 | cost: 0.04066790
epoch:  200/2000 | cost: 0.02513021
epoch:  300/2000 | cost: 0.01552894
epoch:  400/2000 | cost: 0.00959596
epoch:  500/2000 | cost: 0.00592972
epoch:  600/2000 | cost: 0.00366420
epoch:  700/2000 | cost: 0.00226424
epoch:  800/2000 | cost: 0.00139917
epoch:  900/2000 | cost: 0.00086461
epoch: 1000/2000 | cost: 0.00053427
epoch: 1100/2000 | cost: 0.00033015
epoch: 1200/2000 | cost: 0.00020401
epoch: 1300/2000 | cost: 0.00012606
epoch: 1400/2000 | cost: 0.00007790
epoch: 1500/2000 | cost: 0.00004814
epoch: 1600/2000 | cost: 0.00002975
epoch: 1700/2000 | cost: 0.00001838
epoch: 1800/2000 | cost: 0.00001136
epoch: 1900/2000 | cost: 0.00000702
epoch: 2000/2000 | cost: 0.00000434


In [25]:
new_var = torch.FloatTensor([[4.0]])
pred_y = model(new_var)
print(f"input: {new_var.item()}, value: {pred_y}")

input: 4.0, value: tensor([[7.9958]], grad_fn=<AddmmBackward0>)


정답 값인 8에 거의 근접하고 있다

In [26]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9976]], requires_grad=True), Parameter containing:
tensor([0.0055], requires_grad=True)]


forward/backward 연산
1. forward 연산
    - 값 x로부터 예측된 y를 얻는 것
    - 학습 전 x_train을 통해 예측값을 얻는 것
    - 학습이 완료된 후 new_var를 통해 예측값을 얻는 것
    
2. backward 연산
    - 학습 과정에서 비용 함수를 미분하여 기울기을 구하는 것
    - cost.backward() 함수는 비용 함수로부터 기울기를 구하라는 의미로 backward 연산

# 클래스로 파이토치 모델 구현

# 미니 배치와 데이터 로드

# 커스텀 데이터셋